In [1]:
import sys
sys.path.append('../scripts')

from mongo_connection import get_matches_collection
from mongo_connection import get_penalties_collection

collection = get_matches_collection()
drawing = get_penalties_collection()

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [2]:
home = {"$expr": {"$gt": ["$home_score","$away_score"]}}
away = {"$expr": {"$lt": ["$home_score","$away_score"]}}
draw = {"$expr": {"$eq": ["$home_score","$away_score"]}}
homeW = {"winner": "$home_team"}
awayW = {"winner": "$away_team"}
drawE = {"winner": "Draw"}

collection.aggregate([{"$match": home}, {"$addFields": homeW}, {"$merge": "football_Matches"}])
collection.aggregate([{"$match": away}, {"$addFields": awayW}, {"$merge": "football_Matches"}])
collection.aggregate([{"$match": draw}, {"$addFields": drawE}, {"$merge": "football_Matches"}])

In [3]:
from pymongo import ReturnDocument
homes = list(drawing.find({},{"_id": 0, "home_team": 1}))
aways = list(drawing.find({},{"_id": 0, "away_team": 1}))
victors = list(drawing.find({},{"_id": 0, "winner": 1}))
dates = list(drawing.find({}, {"_id": 0, "date": 1}))
z = len(homes)
for x in range(z):
    has = collection.find_one_and_update(
    {
        'home_team': homes[x].get("home_team"),
        'away_team': aways[x].get("away_team"),
        'date': dates[x].get("date")
    },
    {
        "$set": {"winner": victors[x].get("winner")}
    },
    return_document = ReturnDocument.AFTER)